Linear regression is a technique that measures the relationship between two variables. <font color='blue'>If we have an independent variable $X$, and a dependent outcome variable $Y$, linear regression allows us to determine which linear model $Y = \alpha + \beta X$ best explains the data.

<font color='blue'>Python's `statsmodels` library has a built-in linear fit function</font>. 

<font color='blue'>Note that this will give a line of best fit; whether or not the relationship it shows is significant is for you to determine. 

<font color='blue'>The output will also have some statistics about the model, such as R-squared and the F value, which may help you quantify how good the fit actually is.

In [ ]:
from statsmodels import regression
import statsmodels.api as sm

In [ ]:
def linreg(X,Y):
    #/ Running the linear regression
    X = sm.add_constant(X)
    model = regression.linear_model.OLS(Y, X).fit()
    a = model.params[0]
    b = model.params[1]
    X = X[:, 1]
    #/ Return summary of the regression and plot results
    X2 = np.linspace(X.min(), X.max(), 100)
    Y_hat = X2 * b + a
    plt.scatter(X, Y, alpha=0.3) #/ Plot the raw data
    plt.plot(X2, Y_hat, 'r', alpha=0.9);  #/ Add the regression line, colored in red
    plt.xlabel('X Value')
    plt.ylabel('Y Value')
    return model.summary()

In [ ]:
start = '2014-01-01'
end = '2015-01-01'
asset = get_pricing('TSLA', fields='price', start_date=start, end_date=end)
benchmark = get_pricing('SPY', fields='price', start_date=start, end_date=end)
# We have to take the percent changes to get to returns
# Get rid of the first (0th) element because it is NAN
r_a = asset.pct_change()[1:]
r_b = benchmark.pct_change()[1:]
linreg(r_b.values, r_a.values)

As we can see, the line of best fit tells us that for every 1% increased return we see from the SPY, we should see an extra 1.92% from TSLA. This is expressed by the parameter  β , which is 1.9271 as estimated. Of course, for decresed return we will also see about double the loss in TSLA, so we haven't gained anything, we are just more volatile.

<font color='blue'>Linear regression gives us a specific linear model, but is limited to cases of linear dependence.

<font color='blue'>Correlation is general to linear and non-linear dependencies, but doesn't give us an actual model.

<font color='blue'>Both are measures of covariance.

<font color='blue'>Linear regression can give us relationship between Y and many independent variables by making X multidimensional.

<font color='blue'>It is very important to keep in mind that all $\alpha$ and $\beta$ parameters estimated by linear regression are just that - estimates.</font> You can never know the underlying true parameters unless you know the physical process producing the data.

<font color='blue'>The parameters you estimate today may not be the same analysis done including tomorrow's data, and the underlying true parameters may be moving.</font> As such it is very important when doing actual analysis to <font color='blue'>pay attention to the standard error of the parameter estimates.

<font color='blue'>One way to get a sense of how stable your parameter estimates are is to estimate them using a rolling window of data and see how much variance there is in the estimates.

<font color='blue'>The regression model relies on several assumptions:
* The <font color='blue'>independent variable is not random.
* The <font color='blue'>variance of the error term is constant across observations.</font> This is important for evaluating the goodness of the fit.
* The <font color='blue'>errors are not autocorrelated. The Durbin-Watson statistic detects this; if it is close to 2, there is no autocorrelation.
* The <font color='blue'>errors are normally distributed.</font> If this does not hold, we cannot use some of the statistics, such as the F-test.

<font color='blue'>If we confirm that the necessary assumptions of the regression model are satisfied, we can safely use the statistics reported to analyze the fit.

For example, the $R^2$ value tells us the fraction of the total variation of $Y$ that is explained by the model.

<font color='blue'>When making a prediction based on the model, it's useful to report not only a single value but a confidence interval.
    
<font color='blue'>The linear regression reports 95% confidence intervals for the regression parameters,</font> and we can visualize what this means using the `seaborn` library, which plots the regression line and highlights the 95% (by default) confidence interval for the regression line:

In [ ]:
seaborn.regplot(r_b.values, r_a.values);

Regression works by optimizing the placement of the line of best fit (or plane in higher dimensions). It does so by defining how bad the fit is using an objective function. 

<font color='blue'>In ordinary least squares regression (OLS), what we use here, the objective function is:
$$\sum_{i=1}^n (Y_i - a - bX_i)^2$$

We use $a$ and $b$ to represent the potential candidates for $\alpha$ and $\beta$.

Regression is a simple case of numerical optimization that has a closed form solution and does not need any optimizer. We just find the results that minimize the objective function.

<font color='blue'>We will denote the eventual model that results from minimizing our objective function as:
$$ \hat{Y} = \hat{\alpha} + \hat{\beta}X $$

With $\hat{\alpha}$ and $\hat{\beta}$ being the chosen estimates for the parameters that we use for prediction and $\hat{Y}$ being the predicted values of $Y$ given the estimates.

<font color='blue'>We can also find the standard error of estimate, which measures the standard deviation of the error term $\epsilon$, by getting the `scale` parameter of the model returned by the regression and taking its square root.

<font color='blue'>The formula for standard error of estimate is
$$ s = \left( \frac{\sum_{i=1}^n \epsilon_i^2}{n-2} \right)^{1/2} $$

If $\hat{\alpha}$ and $\hat{\beta}$ were the true parameters ($\hat{\alpha} = \alpha$ and $\hat{\beta} = \beta$), we could represent the error for a particular predicted value of $Y$ as $s^2$ for all values of $X_i$. We could simply square the difference $(Y - \hat{Y})$ to get the variance because $\hat{Y}$ incorporates no error in the parameter estimates themselves.

Because $\hat{\alpha}$ and $\hat{\beta}$ are merely estimates in our construction of the model of $Y$, any predicted values , $\hat{Y}$, will have their own standard error based on the distribution of the $X$ terms that we plug into the model.

<font color='blue'>This forecast error is represented by the following:
$$ s_f^2 = s^2 \left( 1 + \frac{1}{n} + \frac{(X - \mu_X)^2}{(n-1)\sigma_X^2} \right) $$

 where $\mu_X$ is the mean of our observations of $X$ and $\sigma_X$ is the standard deviation of $X$. 

<font color='blue'>This adjustment to $s^2$ incorporates the uncertainty in our parameter estimates.</font> Then the 95% confidence interval for the prediction is $\hat{Y} \pm t_cs_f$, where $t_c$ is the critical value of the t-statistic for $n$ samples and a desired 95% confidence.